<a href="https://colab.research.google.com/github/Udana-Gits/My_AI_Learnings/blob/main/pizza_mas_swrl_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pizza MAS with Ontology & SWRL (Notebook)

This notebook reproduces the `pizza_mas_swrl.py` demo as runnable cells with explanations. Requirements: `owlready2`, `mesa`. Run cells sequentially.

## Code

Explanation for code.

In [2]:
# -*- coding: utf-8 -*-
"""
pizza_mas_swrl.py as Jupyter Notebook version
---------------------------------------------
This notebook contains a fully functional ontology-driven multi-agent system (MAS)
for a Pizza shop, using Owlready2 for ontology + SWRL and Mesa for agent simulation.
Sections: 1) Setup & imports, 2) Ontology definition, 3) Individuals, 4) SWRL rules,
5) Helper functions & reasoner, 6) Message bus, 7) MAS model & agents, 8) Run simulation,
9) Inspect OWL messages & summary.
"""

'\npizza_mas_swrl.py as Jupyter Notebook version\n---------------------------------------------\nThis notebook contains a fully functional ontology-driven multi-agent system (MAS)\nfor a Pizza shop, using Owlready2 for ontology + SWRL and Mesa for agent simulation.\nSections: 1) Setup & imports, 2) Ontology definition, 3) Individuals, 4) SWRL rules,\n5) Helper functions & reasoner, 6) Message bus, 7) MAS model & agents, 8) Run simulation,\n9) Inspect OWL messages & summary.\n'

In [5]:
pip install mesa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.1 MB/s eta 0:00:00


In [7]:
pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 45.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.48-cp312-cp312-linux_x86_64.whl size=24545947 sha256=97cfdd1194848b75476aa5e3efef61b83e4cc0689c3a4b3d0700ab1dfb27ed70
  Stored in directory: /root/.cache/pip/wheels/80/6b/6e/3a4fd869625821d573b27fc501b24fd715ab53ee483e981664
Successfully built owlready2


In [8]:
# @title Default title text
#Imports
#from typing import List, Optional
import random
from mesa import Agent, Model
from owlready2 import (
    World, Thing, ObjectProperty, DataProperty, Not,
    Imp, sync_reasoner_pellet
)

## Section 1 — Imports & Setup

Explanation for section 1 — imports & setup.

## Section 2 — Ontology Definition

Explanation for section 2 — ontology definition.

In [9]:
#Build ontology in memory
world = World()
onto = world.get_ontology("http://example.org/pizza.owl")

with onto:
    # Domain concepts
    # ... means Ellipsis literal. Still the complete code is not provided, only the definition.
    class Pizza(Thing): ...
    class Topping(Thing): ...
    class MeatTopping(Topping): ...
    class VegTopping(Topping): ...

    class hasTopping(ObjectProperty):
        domain = [Pizza]
        range = [Topping]

    # VegetarianPizza: Pizza but NOT(hasTopping some MeatTopping)
    class VegetarianPizza(Pizza):
        equivalent_to = [Pizza & Not(hasTopping.some(MeatTopping))]
        pass

    # Agent roles
    class AgentThing(Thing): ...
    class Chef(AgentThing): ...
    class Customer(AgentThing): ...
    class Courier(AgentThing): ...
    class Dispatcher(AgentThing): ...

    # Tasks (kept simple; they showcase role semantics)
    class Task(Thing): ...
    class OrderPizzaTask(Task): ...
    class BakePizzaTask(Task): ...
    class DeliverPizzaTask(Task): ...
    class DispatchTask(Task): ...

    class hasTask(ObjectProperty):
        domain = [AgentThing]
        range  = [Task]

    # Chef constraint: vegetarianOnly = True means can bake only vegetarian pizzas
    class vegetarianOnly(DataProperty):
        domain = [Chef]
        range  = [bool]

    # Messages (OWL individuals for semantic communications)
    class Message(Thing): ...
    class OrderRequest(Message): ...
    class BakeOrder(Message): ...
    class DeliveryOrder(Message): ...

    class sender(ObjectProperty):
        domain = [Message]
        range  = [AgentThing]
    class receiver(ObjectProperty):
        domain = [Message]
        range  = [AgentThing]
    class aboutPizza(ObjectProperty):
        domain = [Message]
        range  = [Pizza]
    class quantity(DataProperty):
        domain = [Message]
        range  = [int]

## Section 3 — Create Pizzas & Toppings (Individuals)

Explanation for section 3 — create pizzas & toppings (individuals).

In [10]:
#Create toppings and pizzas - Individual creations/mappings
with onto:
    tomato = onto.VegTopping("TomatoSauce")
    mozzarella = onto.VegTopping("Mozzarella")
    basil = onto.VegTopping("Basil")
    pepperoni = onto.MeatTopping("Pepperoni")
    chicken = onto.MeatTopping("Chicken")

    margherita = onto.Pizza("MargheritaPizza")
    # mapping individuals
    margherita.hasTopping = [tomato, mozzarella, basil]

    pepperoni_pizza = onto.Pizza("PepperoniPizza")
    pepperoni_pizza.hasTopping = [tomato, mozzarella, pepperoni]

    bbq_chicken = onto.Pizza("BBQChickenPizza")
    bbq_chicken.hasTopping = [tomato, mozzarella, chicken]

## Section 4 — Agent Individuals

Explanation for section 4 — agent individuals.

In [11]:
# Create OWL agent individuals
with onto:
    cust1_ind = onto.Customer("cust1")
    cust1_ind.hasTask = [onto.OrderPizzaTask()]

    chefVeg_ind = onto.Chef("chefVeg")
    chefVeg_ind.hasTask = [onto.BakePizzaTask()]
    chefVeg_ind.vegetarianOnly = [True]

    chefAll_ind = onto.Chef("chefAll")
    chefAll_ind.hasTask = [onto.BakePizzaTask()]
    chefAll_ind.vegetarianOnly = [False]

    courier1_ind = onto.Courier("courier1")
    courier1_ind.hasTask = [onto.DeliverPizzaTask()]

    dispatcher_ind = onto.Dispatcher("dispatcher1")
    dispatcher_ind.hasTask = [onto.DispatchTask()]

## Section 5 — SWRL Rules

Explanation for section 5 — swrl rules. - Handle message routing logics

In [12]:
#SWRL Rules (vegetarian routing)
# If a message is an OrderRequest about a VegetarianPizza, then assign its receiver as chefVeg
with onto:
    rule_route_veg = Imp()

    rule_route_veg.set_as_rule(f"""
        OrderRequest(?m) ^ aboutPizza(?m, ?p) ^ VegetarianPizza(?p) -> receiver(?m, {chefVeg_ind.name})
    """)

In [13]:
#SWRL Rules (non-vegetarian routing)
# If a message is an OrderRequest about a non- VegetarianPizza, then assign its receiver as chefNon-Veg
with onto:
    rule_nonveg = Imp()

rule_nonveg.set_as_rule(f"""
    OrderRequest(?m) ^ aboutPizza(?m, ?p) ^ hasTopping(?p, ?t) ^ MeatTopping(?t)
    -> receiver(?m, {chefAll_ind.name})
""")


# If it’s a pizza that’s **not vegetarian**, send it to the general chef.


OrderRequest(?m), aboutPizza(?m, ?p), hasTopping(?p, ?t), MeatTopping(?t) -> receiver(?m, pizza.chefAll)

## Section 6 — Reasoner Helpers

Explanation for section 6 — reasoner helpers.

In [14]:
#Reasoner helpers
# If needed string param can be provided. (optional)
# load the reasoner.
def try_reason(label:str=""):
    try:
        # infer the propertiese as well, not the classes only
        sync_reasoner_pellet(infer_property_values=True, debug=0)
        if label:
            print(f"[Reasoner] Pellet inference ran ({label}).")
        else:
            print("[Reasoner] Pellet inference ran.")
        return True
    except Exception as e:
        print(f"[Reasoner] Skipped (Pellet/Java not available): {e}")
        return False
# returns true if the pizza_ind param captured is a vegetarian individual
def is_vegetarian(pizza_ind) -> bool:
    try:
        return onto.VegetarianPizza in pizza_ind.is_a
        #in case if reasoner failed, check for pizzas manually which is not with a meat topping
    except Exception:
        return all(not isinstance(t, onto.MeatTopping) for t in pizza_ind.hasTopping)

## Section 7 — Message Bus

Explanation for section 7 — message bus.

In [16]:
#Python-side message bus
# responsible for initializing the msg object
class BusMessage:
    def __init__(self, owl_ind, mtype: str, sender_id: str, receiver_id: Optional[str], pizza_ind, qty: int):
        self.owl_ind = owl_ind
        self.type = mtype
        self.sender_id = sender_id
        self.receiver_id = receiver_id
        self.pizza = pizza_ind
        self.qty = qty
        self.processed = False
# showcase the initialized msg object contents
    def __repr__(self):
        return f"<Msg {self.type} {self.pizza.name} x{self.qty} {self.sender_id}->{self.receiver_id}>"

NameError: name 'Optional' is not defined

## Section 8 — MAS Model & Agents

Explanation for section 8 — mas model & agents.

In [ ]:
#MAS model & agents
class PizzaMAS(Model):
    def __init__(self):
        super().__init__()
        # setting up the propertiese
        self.messages: List[BusMessage] = []
        self.baked = 0
        self.delivered = 0
        self.reasoner_available = False
        # initializing the individuals
        self.cust = cust1_ind
        self.chefVeg = chefVeg_ind
        self.chefAll = chefAll_ind
        self.courier = courier1_ind
        self.dispatcher = dispatcher_ind
        # mapping names for the individuals
        self.agent_customer = CustomerAgent("cust1", self, self.cust)
        self.agent_dispatcher = DispatcherAgent("dispatcher1", self, self.dispatcher)
        self.agent_chefVeg = ChefAgent("chefVeg", self, self.chefVeg)
        self.agent_chefAll = ChefAgent("chefAll", self, self.chefAll)
        self.agent_courier = CourierAgent("courier1", self, self.courier)

        self._customer_has_ordered = False
        self.reasoner_available = try_reason("initial")

        print("\\n[Ontology] Vegetarian classification:")
        # 18 is formatting the spaces when printing to get a neat o/p.
        for p in onto.Pizza.instances():
            print(f" - {p.name:18} vegetarian={is_vegetarian(p)}")
    # compilation of tasks msgs as required
    def send_order(self, sender_name:str, pizza_ind, qty:int=1):
        with onto:
            owl_msg = onto.OrderRequest(f"order_{sender_name}_{random.randrange(1_000_000)}")
            owl_msg.sender = [getattr(onto, sender_name)]
            owl_msg.aboutPizza = [pizza_ind]
            owl_msg.quantity = [qty]
        msg = BusMessage(owl_msg, "order", sender_name, None, pizza_ind, qty)
        self.messages.append(msg)
        print(f"[MSG] {msg}")

    def send_bake(self, sender_name:str, receiver_name:str, pizza_ind, qty:int):
        with onto:
            owl_msg = onto.BakeOrder(f"bake_{sender_name}_{random.randrange(1_000_000)}")
            owl_msg.sender = [getattr(onto, sender_name)]
            owl_msg.receiver = [getattr(onto, receiver_name)]
            owl_msg.aboutPizza = [pizza_ind]
            owl_msg.quantity = [qty]
        msg = BusMessage(owl_msg, "bake", sender_name, receiver_name, pizza_ind, qty)
        self.messages.append(msg)
        print(f"[MSG] {msg}")

    def send_delivery(self, sender_name:str, receiver_name:str, pizza_ind, qty:int):
        with onto:
            owl_msg = onto.DeliveryOrder(f"deliv_{sender_name}_{random.randrange(1_000_000)}")
            owl_msg.sender = [getattr(onto, sender_name)]
            owl_msg.receiver = [getattr(onto, receiver_name)]
            owl_msg.aboutPizza = [pizza_ind]
            owl_msg.quantity = [qty]
        msg = BusMessage(owl_msg, "deliver", sender_name, receiver_name, pizza_ind, qty)
        self.messages.append(msg)
        print(f"[MSG] {msg}")
    # define the order of the actions to be executed
    def step(self):
        self.agent_customer.step()
        self.reasoner_available = try_reason("per-step")
        self.agent_dispatcher.step()
        self.agent_chefVeg.step()
        self.agent_chefAll.step()
        self.agent_courier.step()
# create agent objects
class OntologyBackedAgent(Agent):
    def __init__(self, unique_id: str, model: PizzaMAS, owl_ind):
        super().__init__(unique_id, model)
        self.owl_ind = owl_ind

class CustomerAgent(OntologyBackedAgent):
    def step(self):
        if self.model._customer_has_ordered:
            return
        pizzas = list(onto.Pizza.instances())
        veg = [p for p in pizzas if is_vegetarian(p)]
        pizza = random.choice(veg if veg else pizzas)
        qty = random.choice([1, 2])
        print(f"[Customer] Ordering {pizza.name} x{qty}")
        self.model.send_order(self.unique_id, pizza, qty)
        self.model._customer_has_ordered = True

class DispatcherAgent(OntologyBackedAgent):
    def step(self):
        for msg in self.model.messages:
            if msg.type != "order" or msg.processed:
                continue
            owlr = getattr(msg.owl_ind, "receiver", [])
            if owlr:
                recv_name = owlr[0].name
                print(f"[DISPATCH] SWRL routed {msg.pizza.name} -> {recv_name}")
                self.model.send_bake(self.unique_id, recv_name, msg.pizza, msg.qty)
                msg.processed = True
                continue
            recv_name = "chefVeg" if is_vegetarian(msg.pizza) else "chefAll"
            print(f"[DISPATCH] Fallback route {msg.pizza.name} -> {recv_name}")
            self.model.send_bake(self.unique_id, recv_name, msg.pizza, msg.qty)
            msg.processed = True

class ChefAgent(OntologyBackedAgent):
    def step(self):
        for msg in self.model.messages:
            if msg.type != "bake" or msg.processed:
                continue
            if msg.receiver_id != self.unique_id:
                continue
            veg_only_vals = getattr(self.owl_ind, "vegetarianOnly", [])
            veg_only = bool(veg_only_vals and veg_only_vals[0])
            if veg_only and not is_vegetarian(msg.pizza):
                print(f"[CHEF {self.unique_id}] Rejects non-veg: {msg.pizza.name}")
                msg.processed = True
                continue
            print(f"[CHEF {self.unique_id}] Baking {msg.pizza.name} x{msg.qty}")
            self.model.baked += msg.qty
            self.model.send_delivery(self.unique_id, "courier1", msg.pizza, msg.qty)
            msg.processed = True

class CourierAgent(OntologyBackedAgent):
    def step(self):
        for msg in self.model.messages:
            if msg.type != "deliver" or msg.processed:
                continue
            if msg.receiver_id != self.unique_id:
                continue
            print(f"[COURIER] Delivered {msg.pizza.name} x{msg.qty}")
            self.model.delivered += msg.qty
            msg.processed = True

## Section 9 — Run Simulation

Explanation for section 9 — run simulation.

In [ ]:
#Run
def main():
    model = PizzaMAS()
    print("\\n=== Simulation Start ===")
    for t in range(5):
        print(f"\\n--- Step {t} ---")
        model.step()
    print("\\n=== Summary ===")
    print(f"Baked: {model.baked}, Delivered: {model.delivered}")
    print("\\n[OWL Messages created]")
    for m in onto.Message.instances():
        mtype = m.is_a[0].name if m.is_a else "Message"
        s = m.sender[0].name if getattr(m, "sender", []) else "?"
        r = m.receiver[0].name if getattr(m, "receiver", []) else "?"
        p = m.aboutPizza[0].name if getattr(m, "aboutPizza", []) else "?"
        q = m.quantity[0] if getattr(m, "quantity", []) else "?"
        print(f" - {m.name}: {mtype} {p} x{q} {s}->{r}")

if __name__ == "__main__":
    main()

[Reasoner] Pellet inference ran (initial).
\n[Ontology] Vegetarian classification:
 - MargheritaPizza    vegetarian=False
 - PepperoniPizza     vegetarian=False
 - BBQChickenPizza    vegetarian=False
\n=== Simulation Start ===
\n--- Step 0 ---
[Customer] Ordering MargheritaPizza x2
[MSG] <Msg order MargheritaPizza x2 cust1->None>
[Reasoner] Pellet inference ran (per-step).
[DISPATCH] Fallback route MargheritaPizza -> chefAll
[MSG] <Msg bake MargheritaPizza x2 dispatcher1->chefAll>
[CHEF chefAll] Baking MargheritaPizza x2
[MSG] <Msg deliver MargheritaPizza x2 chefAll->courier1>
[COURIER] Delivered MargheritaPizza x2
\n--- Step 1 ---
[Reasoner] Pellet inference ran (per-step).
\n--- Step 2 ---
[Reasoner] Pellet inference ran (per-step).
\n--- Step 3 ---
[Reasoner] Pellet inference ran (per-step).
\n--- Step 4 ---
[Reasoner] Pellet inference ran (per-step).
\n=== Summary ===
Baked: 2, Delivered: 2
\n[OWL Messages created]
 - order_cust1_473780: OrderRequest PepperoniPizza x2 cust1->?
 - b